<a href="https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# coding=utf-8
from re import findall
from time import time
from google.colab import output
from IPython.display import display,HTML
from base64 import b64decode
try:
	from selenium import webdriver
except:
	# 清除輸出Log
	print('Install package...\n下載所需插件以便執行...')
	!apt -q -qq update
	!apt -q -qq install chromium-chromedriver
	!pip -q install selenium
	from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
# options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#		 INFO = 0, 
#		 WARNING = 1, 
#		 LOG_ERROR = 2, 
#		 LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
# 清除輸出内容
output.clear()
username = "tiaccwhf.5a29" #@param {type:"string"}
password = "2020" #@param {type:"string"}
lessons_type = "Wiseman" #@param ["Wiseman", "Fun and Friends"]
if lessons_type == "Fun and Friends":
	print('登入Fun and Friends')
	driver.get('https://fnfclub.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	print('進入課程列表')
	driver.get('https://fnfclub.hk/lms/user/secure/course/selector/do/lessons.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="effect_session"]/table[@class="list_line"]/tbody/tr')))
	display(HTML(
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')+
		driver.find_element_by_xpath('//div[@class="effect_session"]').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')
	))
	url=[]
	for tr in driver.find_elements_by_xpath('//div[@class="effect_session"]/table[@class="list_line"]/tbody/tr'):
		if '-' in tr.find_element_by_xpath('.//td[4]').get_attribute('innerText'):
			try:
				href=tr.find_element_by_xpath('.//td[3]/span/a[contains(@href,"/lms/user/secure/course/selector/do/selectLesson.do?id=")]').get_attribute('href')
				url.append(href)
				print(href,end='\r')
			except NoSuchElementException as identifier:
				print('Next',end='\r')
	for url in url:
		driver.get(url)
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		tm=time()
		while time()-tm<=5:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
		token = str(driver.execute_script('return window.location.href;')).replace('https://fnfclub.hk/lms/scorm/?token=','')
		lesson_id = findall(r'id=.*&from=lessons',url)[0].replace('id=','').replace('&from=lessons','')
		print('token:%s'%(token))
		print('lesson_id:%s'%lesson_id)
		driver.get('https://fnfclub.hk/lms/scorm/commit.do?token=%s&data={"wiseman.lesson_id":"%s","cmi.core.lesson_status":"completed","wiseman.playground_score":100,"wiseman_init":"true","cmi.suspend_data":"true","cmi.core.score.raw":100,"wiseman.hostname":"fnfclub.hk","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":"","wiseman.like":"","wiseman.error_rate":"","wiseman.target":"","wiseman.total_like":""}'%(token,lesson_id))
		print('Submited',end='\r')
		tm=time()
		while time()-tm<=20:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
	# 登出Fun and Friends
	driver.get('https://fnfclub.hk/lms/user/logout.do')
elif lessons_type == "Wiseman":
	print('登入Wiseman',end='\r')
	driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	display(HTML(
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+
		driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
		driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]').get_attribute('innerHTML'))#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
	))
	print('進入課程列表',end='\r')
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')))
	display(HTML(driver.find_element_by_xpath('//div[@class="table-responsive"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')))
	url=[]
	for tr in driver.find_elements_by_xpath('//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr'):
		if '-' in tr.find_element_by_xpath('.//td[@class="align-middle text-center"]').get_attribute('innerText'):
			try:
				href=tr.find_element_by_xpath('.//a[contains(@href,"/lms/user/secure/course/eb/select_theme/selectLesson")]').get_attribute('href')
				url.append(href)
				print(href,end='\r')
			except NoSuchElementException as identifier:
				print('Next',end='\r')
	for url in url:
		driver.get(url)
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		tm=time()
		while time()-tm<=5:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
		token = str(driver.execute_script('return window.location.href;')).replace('https://lms.wiseman.com.hk/lms/scorm/?token=','')
		print('token:%s'%(token))
		# driver.execute_script('window.location.href=\'https://lms.wiseman.com.hk/lms/scorm/commit.do?token=%s&data={"cmi.core.lesson_status":"completed","wiseman.playground_score":"0","wiseman.like":"0","wiseman_init":"true","wiseman.error_rate":"","wiseman.target":"-1","cmi.suspend_data":"true","cmi.core.score.raw":100,"wiseman.total_like":"0","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":""}\';'%(token))
		driver.get('https://lms.wiseman.com.hk/lms/scorm/commit.do?token=%s&data={"cmi.core.lesson_status":"completed","wiseman.playground_score":"0","wiseman.like":"0","wiseman_init":"true","wiseman.error_rate":"","wiseman.target":"-1","cmi.suspend_data":"true","cmi.core.score.raw":100,"wiseman.total_like":"0","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":""}'%(token))
		print('Submited',end='\r')
		tm=time()
		while time()-tm<=20:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
	# 登出Wiseman
	driver.get('https://lms.wiseman.com.hk/lms/user/logout.do')
driver.quit()

登入Wiseman
Welcome! 5A-21-NG BING FAI 
Your Score:1400
進入課程列表


Date,Theme / ModuleLesson Name,Status,Score,Expiry Date
14 Sep (Mon),Short Stories / Detectives and Crime Detectives and Dubai - Listening 0,,100,30 Jun (Wed)
14 Sep (Mon),Short Stories / Detectives and Crime Detectives and Dubai - Reading 0,,100,30 Jun (Wed)
21 Sep (Mon),Short Stories / Detectives and Crime Detectives and Dubai - Speaking 0,,100,30 Jun (Wed)
21 Sep (Mon),Short Stories / Detectives and Crime Detectives and Dubai - Writing 0,,100,30 Jun (Wed)
28 Sep (Mon),Technology / Rise of the Robots Can Machines Have Feelings - Listening 0,,100,30 Jun (Wed)
28 Sep (Mon),Technology / Rise of the Robots Can Machines Have Feelings - Reading 0,,100,30 Jun (Wed)
5 Oct (Mon),Technology / Rise of the Robots Can Machines Have Feelings - Speaking 0,,100,30 Jun (Wed)
5 Oct (Mon),Technology / Rise of the Robots Can Machines Have Feelings - Writing 0,,100,30 Jun (Wed)
12 Oct (Mon),Hot Issues / I Wonder... How Smart Are Animals - Listening 0,,-,30 Jun (Wed)
12 Oct (Mon),Hot Issues / I Wonder... How Smart Are Animals - Reading 0,,-,30 Jun (Wed)


token:53a3fc585ae92df1ff1bcee39587e422875a36ceea235795ecb9144638e6180a
token:9f44306ea08a1d90c22ab1da39f9ba0a533432af97d18c9a88c1c0dc24e6e553
token:3e20c3b37dbe0f3abdc7ec82244408ad97eade7f247c4ff6f0240e8a840b9318
token:b16df1379042e012034ddeb34702d0920b225c7cd5e31b7a6fc161874fda54a0
token:52db80a1a7408545f8623f3ebef4a2e9a777e47eb8cdf0b26692b5a186564dbf
token:2cb134bb81e0e24fdf0c947c97b0d37bdf64526e93fbac0ebbe5f9b06c5a895a
token:d5610c6661aefb95994541399967c4a3de17f24f0718d974f2f3d907dbfc9c3c
token:c74bbabb4cd7a02576b97a0f9d97ffa5987ca4c0b917a244818bdb58a20a17ea
token:4b6499e2f5693b33abd63d45b0178edb690445c0fa9e40f86c5071bdfe3d9950
token:77fcb6c462315da011aa981cab31586108fdd0832a10ca8df0855a80e1b4a230
token:66fa74af1aa78b4274486e00b7de610527bdf723ec6970456c8ff54188ad1d2c
token:2a9800ea9ce4cf7c8bfe6270f6afedcd2f274035d9b36926333b553679e98991
token:b87576e66d32edc470a778be9e18069c93380a5754289272cb787a9f6d1f277a
token:f8446262733cc73c44c6ecaaa65bc4fc6845e8346e5071dd66ce0b87d5713c7e
